## XGBoost Model for Time Series Prediction
#### Group Project of Information Retrieval and Data Mining 2016 @ UCL

Authors: [Yijing Yang](yijing.yang.15@ucl.ac.uk), Xinyi He, Ying Wen

This notebook trains a XGBoost model for load forcasting on [Global Energy Forecasting Competition 2012](https://www.kaggle.com/c/global-energy-forecasting-competition-2012-load-forecasting) dataset with [XGBoost](https://github.com/dmlc/xgboost).

Except where otherwise noted, this work is subject to a Creative Common Attribution-NonCommercial 4.0 License. [Details.](https://creativecommons.org/licenses/by-nc/4.0/)



In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import operator
import xgboost as xgb
from sklearn.metrics import mean_squared_error

## Data Processing

In [2]:
data_load = pd.read_csv('/Users/yangyijing/Desktop/IR_group/data/full_load.csv',header = None)
data_load.columns = ['zone1','zone2','zone3','zone4','zone5','zone6','zone7','zone8','zone9','zone10','zone11','zone12','zone13','zone14','zone15','zone16','zone17','zone18','zone19','zone20']
data_load = np.log(data_load + 1)
time = pd.read_csv('/Users/yangyijing/Desktop/IR_group/data/time_load.csv',header = None)
time.columns = ['year','month','day','hour'] 

In [6]:
for i in range(0,len(time)):
    if time.ix[i,0] == 0:
        time.ix[i,0] = '2004'
    if time.ix[i,0] == 1:
        time.ix[i,0] = '2005'
    if time.ix[i,0] == 2:
        time.ix[i,0] = '2006'
    if time.ix[i,0] == 3:
        time.ix[i,0] = '2007'
    if time.ix[i,0] == 4:
        time.ix[i,0] = '2008'
time1 = time
time1['date'] = time1.apply(lambda x:pd.datetime.strptime("{0} {1} {2}".format(x['year'],x['month'], x['day']), '%Y %m %d'),axis=1)
time1['date_time'] = pd.to_datetime(time1['date']) + pd.TimedeltaIndex(time1['hour'], unit='H')
load_withtime = pd.concat([time1,data_load],axis = 1)

In [10]:
temp = pd.read_csv('/Users/yangyijing/Desktop/IR_group/data/full_temp.csv',header = None)
temp.columns =['station1','station2','station3','station4','station5','station6','station7','station8','station9','station10','station11']
load_withtime = pd.concat([load_withtime,temp],axis = 1)

#### Split the records based on zone_id

This step is the preparation for building time lag features. We split records based on zone_id and generate a new column called 'zone_id'.

In [13]:
def data_zone(i):
    t = load_withtime.ix[:,[0,1,2,3,4,5,5+i,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]
    t.columns = ['year','month','day','hour','date','date_time','load','station1','station2','station3','station4','station5','station6','station7','station8','station9','station10','station11'] 
    s = [i] * len(t)
    t['zone_id'] = s
    load = pd.Series(t['load'])
    lagload = pd.concat([load.shift(j) for j in range(1,25)],axis=1)
    lagload.columns = ['lag_t-%d' %j for j in range(1,25)]
    load_lag = pd.concat([t, lagload], axis=1)
    l = len(load_lag)
    load_lag = load_lag.tail(l - 24)
    return load_lag

In [14]:
t1 = data_zone(1)
t2 = data_zone(2)
t3 = data_zone(3)
t4 = data_zone(4)
t5 = data_zone(5)
t6 = data_zone(6)
t7 = data_zone(7)
t8 = data_zone(8)
t9 = data_zone(9)
t10 = data_zone(10)
t11 = data_zone(11)
t12 = data_zone(12)
t13 = data_zone(13)
t14 = data_zone(14)
t15 = data_zone(15)
t16 = data_zone(16)
t17 = data_zone(17)
t18 = data_zone(18)
t19 = data_zone(19)
t20 = data_zone(20)
combinezones = pd.concat([t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20],axis = 0)

/Users/yangyijing/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
df = combinezones[combinezones['zone_id'] == 1]
df = df.head(39222)
for i in range(2,21):
    df1 = combinezones[combinezones['zone_id'] == i]
    df1 = df1.head(39222)
    df = pd.concat([df,df1],axis = 0)
    

#### Feature importance

Now, we have lagged loads based on 24 hours as new features. We use XGBoost to test the feature importance of the 24 features. As a result, we can find several most important lag features.

In [18]:
def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

In [19]:
def get_data(df):
    train = df
    features = list(df.columns[19:])
    y_train = train.load
    x_train = train[features]

    return features, x_train, y_train

In [20]:
features, x_train, y_train = get_data(df)
ceate_feature_map(features)

xgb_params = {"objective": "reg:linear", "eta": 0.01, "max_depth": 8, "seed": 42, "silent": 1}
num_rounds = 500

dtrain = xgb.DMatrix(x_train, label=y_train)
gbdt = xgb.train(xgb_params, dtrain, num_rounds)

importance = gbdt.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')

#### Transforming categorical variables

After feature importance, we select lag1,2,3,24 as the important lag features. Then we need to transform categorical variables,year,month,day,hour,zone_id, to dummies.

In [21]:
data = combinezones.ix[:,[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,-1]].reset_index()
data = data.drop('index',axis = 1)

In [23]:
# Create and merge binary year features
predictBy=1
predictBy = -(predictBy-1)
year = pd.get_dummies(pd.DatetimeIndex(data.date_time).year, prefix='Year')
year = year.set_index(pd.DatetimeIndex(data.date_time))
if predictBy != 0:
        if predictBy > 0 or type(predictBy) != int:
            raise ValueError("predictBy must be greater than or equal to 1 and an int. Found predictBy={} and type={}"
                             .format(-(predictBy+1), type(predictBy)))
        else:
            year = year.shift(predictBy)
data = data.set_index('date_time')
data = pd.concat([data, year], axis=1)

In [25]:
# Create and merge binary month features
months = pd.get_dummies(pd.DatetimeIndex(data.index).month, prefix='Month')
months = months.set_index(pd.DatetimeIndex(data.index))
if predictBy != 0:
    if predictBy > 0 or type(predictBy) != int:
        raise ValueError("predictBy must be greater than or equal to 1 and an int. Found predictBy={} and type={}"
                             .format(-(predictBy+1), type(predictBy)))
    else:
        months = months.shift(predictBy)
data = pd.concat([data, months], axis=1)

In [27]:
# Create and merge binary day features
days = pd.get_dummies(pd.DatetimeIndex(data.index).day, prefix='Day')
days = days.set_index(pd.DatetimeIndex(data.index))
if predictBy != 0:
    if predictBy > 0 or type(predictBy) != int:
        raise ValueError("predictBy must be greater than or equal to 1 and an int. Found predictBy={} and type={}"
                             .format(-(predictBy+1), type(predictBy)))
    else:
        days = days.shift(predictBy)
data = pd.concat([data, days], axis=1)

In [29]:
# Create and merge binary hour features
hours = pd.get_dummies(pd.DatetimeIndex(data.index).hour, prefix='Hour')
hours = hours.set_index(pd.DatetimeIndex(data.index))
if predictBy != 0:
    if predictBy > 0 or type(predictBy) != int:
        raise ValueError("predictBy must be greater than or equal to 1 and an int. Found predictBy={} and type={}"
                             .format(-(predictBy+1), type(predictBy)))
    else:
        hours = hours.shift(predictBy)
data = pd.concat([data, hours], axis=1)

In [31]:
# Create and merge binary zone features
zones = pd.get_dummies(data.zone_id,prefix='Zone')
if predictBy != 0:
    if predictBy > 0 or type(predictBy) != int:
        raise ValueError("predictBy must be greater than or equal to 1 and an int. Found predictBy={} and type={}"
                             .format(-(predictBy+1), type(predictBy)))
    else:
        zones = zones.shift(predictBy)
data = pd.concat([data, zones], axis=1)

generate the actual data(records from 23/06/2008 to 30/06/2008) for comparison with our predictions

In [323]:
data_test = data_load.tail(354).head(168)
data_test = np.exp(data_test) - 1
data_test = data_test.reset_index()
data_test = data_test.drop('index',axis = 1)

## Training Model

#### Matching temperatures of stations with zones

This part is used to match the stations with zones. We train XGBoost model for each zones with all 11 stations. For each time with one station, we get MSE when comparing predictions with acutal data. We consider the stataion with the least MSE as the correct station for the zone and use the temperature of this station to predict the load of this zone.

In [175]:
def data_xgboost(station_id,zone_id):
    train = data[data['zone_id'] == zone_id]
    features = list(data.columns[17:])
    features.append(data.columns[station_id+4])
    Y = train.load
    X = train[features]
    X_train = X.head(39222)
    Y_train = Y.head(39222)
    X_test = X.tail(354+168).head(168+168)
    X_test = X_test.tail(len(X_test)-168)
    X_testlag = data.tail(354+168).head(168+168)
    return X_train,Y_train,X_testlag,X_test

In [176]:
def prediction(index,X_train,Y_train,X_testlag,X_test,gbdt):
    dtest = xgb.DMatrix(X_test)
    ypred = gbdt.predict(dtest)
    ypred = ypred[index]
    X_testlag.ix[168 + index,4] = ypred
    lag = X_testlag['load']
    load = pd.Series(lag)
    lagload = pd.concat([load.shift(j) for j in [1,2,3,24]],axis=1)
    lagload.columns = ['lag_t-%d' %j for j in [1,2,3,24]]
    load_lag = pd.concat([load,lagload], axis=1)
    test = X_test
    load_lag= load_lag.tail(len(load_lag)-168)
    test['lag_t-1'] = load_lag['lag_t-1']
    test['lag_t-2'] = load_lag['lag_t-2']
    test['lag_t-3'] = load_lag['lag_t-3']
    test['lag_t-24'] = load_lag['lag_t-24']
    return ypred,test,X_testlag


In [177]:
def zone_station(station_id,zone_id):
    X_train,Y_train,X_testlag,X_test = data_xgboost(station_id,zone_id)
    xgb_params = {"objective": "reg:linear", "eta": 0.01, "max_depth": 8, "seed": 42, "silent": 1}
    num_rounds = 1000
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    gbdt = xgb.train(xgb_params, dtrain, num_rounds)
    pre = []
    for i in range(0,168):  
        ypred,test,testlag = prediction(i,X_train,Y_train,X_testlag,X_test,gbdt)
        ypred = np.exp(ypred) - 1
        pre.append(ypred)
        X_testlag = testlag
        X_test = test
    return pre

In [178]:
def fit_zone(zone_id):
    mse = []
    for i in range(1,12):
        pre = zone_station(i,zone_id)
        m = ((pre - data_test.ix[:,zone_id-1]) ** 2).mean()
        mse.append(m)
    return mse

In [179]:
mse1 = fit_zone(1)

In [180]:
mse1

[21270952.80542777,
 3927653.8992050355,
 18121670.56752389,
 22002925.579662774,
 19475431.258014724,
 8419631.395374523,
 12542541.833644276,
 18832205.3362693,
 15809877.502839724,
 9878435.277929336,
 11106709.427944656]

In [181]:
mse2 = fit_zone(2)
mse2

[392999315.4694524,
 249274255.8481766,
 280670680.9364505,
 420464942.50753546,
 362733863.32131356,
 221252753.96372646,
 284793706.56537443,
 305123177.8136957,
 287098076.3789874,
 237228090.66902184,
 206671079.1043431]

In [182]:
mse3 = fit_zone(3)
mse3

[433609966.2443199,
 274159245.70301276,
 318407590.9105416,
 485972255.21537143,
 497630909.7235389,
 255603428.42413068,
 351064111.7350747,
 383132785.5359805,
 357854719.89796513,
 273755473.738232,
 246250869.90055284]

In [243]:
mse4 = fit_zone(4)
mse4

[155327.34163537849,
 168043.3274320794,
 156496.7434227795,
 163957.2337537399,
 151713.83484553505,
 168533.45339572334,
 168609.24686965984,
 165972.18801073532,
 147877.59308559677,
 169944.55046247618,
 165121.26244331436]

In [244]:
mse5 = fit_zone(5)
mse5

[3107361.7154635256,
 1711032.8410049293,
 2186285.1662513707,
 3543525.8156238594,
 2413399.3343559727,
 2562802.997875697,
 1590116.060498284,
 3155075.558573044,
 2162558.3221690455,
 2516188.5055949907,
 1342579.7630209592]

In [245]:
mse6 = fit_zone(6)
mse6

[473633123.9924499,
 263592376.41835228,
 269190145.73303205,
 501647250.1424596,
 455931423.42795354,
 260082638.10179323,
 390149162.7630609,
 331033477.9336039,
 350951883.85667086,
 248267388.66798392,
 159214491.42981917]

In [246]:
mse7 = fit_zone(7)
mse7

[433609966.2443199,
 274159245.70301276,
 318407590.9105416,
 485972255.21537143,
 497630909.7235389,
 255603428.42413068,
 351064111.7350747,
 383132785.5359805,
 357854719.89796513,
 273755473.738232,
 246250869.90055284]

In [247]:
mse8 = fit_zone(8)
mse8

[326985.0045926078,
 157647.8759511372,
 268429.33310787333,
 328184.81575939927,
 294851.3494319672,
 253689.18290295775,
 205091.79513037528,
 252298.05897135075,
 214043.941442838,
 237374.21393530504,
 175156.47503426313]

In [248]:
mse9 = fit_zone(9)
mse9

[318402756.371569,
 310295108.65602887,
 377846523.96056557,
 291659701.4955377,
 311736911.91899586,
 289928971.03480136,
 326421974.8833237,
 329569987.6404961,
 468226531.81805956,
 292716362.3081562,
 382525882.9385512]

In [249]:
mse10 = fit_zone(10)
mse10

[2138115430.3176327,
 2647096352.0681014,
 2617999483.9701447,
 2555747620.3655305,
 2658368073.8178506,
 2665798574.7475104,
 2579333664.600109,
 2601950167.214591,
 2662031569.4605217,
 2638980411.2825646,
 2671652631.333048]

In [250]:
mse11 = fit_zone(11)
mse11

[182652317.50432193,
 1559040211.8469188,
 190481363.29324314,
 339817228.66322994,
 291802216.56015,
 605460607.5666136,
 642436518.166125,
 374915100.22330266,
 454979694.24576515,
 586971237.9855677,
 1352435109.2712781]

In [251]:
mse12 = fit_zone(12)
mse12

[816121302.622349,
 4602955426.223239,
 373901565.0030951,
 1719146934.311291,
 636070932.5243165,
 3405121294.1647573,
 2251249556.0268316,
 1551764031.9099965,
 1923317299.3634615,
 2753304487.227988,
 3803719432.358491]

In [252]:
mse13 = fit_zone(13)
mse13

[21025961.783017185,
 6465778.65945715,
 17461109.56695033,
 22646926.402513135,
 17628211.841774818,
 11653729.483474161,
 10061744.709833344,
 19228820.489822112,
 13342301.722694837,
 10441142.012077795,
 7005695.559240286]

In [253]:
mse14 = fit_zone(14)
mse14

[7254727.993312182,
 21780740.72786538,
 8050108.445405632,
 6293393.204260754,
 8420109.938435014,
 7776772.407155903,
 8339379.877752161,
 6794406.074812086,
 7309943.16821764,
 8491224.298802694,
 15355356.231732098]

In [254]:
mse15 = fit_zone(15)
mse15

[239581180.54342836,
 281307666.60536397,
 242281771.8825172,
 163857704.08976027,
 281456200.6455739,
 164195332.0316177,
 224540483.3030723,
 142981255.08335012,
 240972218.8831353,
 149923900.44419545,
 344537369.0185592]

In [255]:
mse16 = fit_zone(16)
mse16

[9030183.883793607,
 57717641.66828687,
 7087420.662493124,
 11395228.724334737,
 7772363.263351134,
 21783161.113339756,
 15589746.281293914,
 9428809.056999117,
 11032435.793432236,
 18358360.337781712,
 33793895.04063553]

In [256]:
mse17 = fit_zone(17)
mse17

[11079495.08232228,
 15828984.768230217,
 9785031.754987868,
 8758832.698415779,
 11899343.973843459,
 5424060.033943743,
 8119351.585552146,
 6859097.070451848,
 8065537.535505201,
 5703442.744417916,
 14214847.068800347]

In [257]:
mse18 = fit_zone(18)
mse18

[1438385165.849941,
 3077555521.2414136,
 1302939299.8252237,
 2140505312.558862,
 2038107924.5151875,
 1872511059.3027067,
 1999377143.3373427,
 1691966895.3661199,
 1955689498.2139997,
 1993838400.5300016,
 2913478856.522496]

In [258]:
mse19 = fit_zone(19)
mse19

[583721115.6792753,
 647232114.556185,
 568582995.7399657,
 525984584.73740286,
 640658283.5762403,
 366102864.1813942,
 455674934.63970554,
 413035240.37604994,
 497307390.45445824,
 388109017.3767198,
 726446323.200489]

In [259]:
mse20 = fit_zone(20)
mse20

[3280938785.6571865,
 3023055272.304011,
 3259010216.7908034,
 3070343253.6242156,
 3300624823.0070605,
 3182229323.254192,
 3143042210.8285456,
 3216596835.239583,
 3259491106.3356524,
 3172313365.5962057,
 2757424096.696284]

## Testing

#### After matching the stations and zones, we use the corresponding temperatures as one of the features for predictions of testing data.

In [260]:
pre1 = zone_station(2,1)
pre2 = zone_station(11,2)
pre3 = zone_station(11,3)
pre4 = zone_station(9,4)
pre5 = zone_station(11,5)
pre6 = zone_station(11,6)
pre7 = zone_station(11,7)
pre8 = zone_station(2,8)
pre9 = zone_station(6,9)
pre10 = zone_station(1,10)
pre11 = zone_station(1,11)
pre12 = zone_station(3,12)
pre13 = zone_station(2,13)
pre14 = zone_station(4,14)
pre15 = zone_station(8,15)
pre16 = zone_station(3,16)
pre17 = zone_station(6,17)
pre18 = zone_station(3,18)
pre19 = zone_station(6,19)
pre20 = zone_station(11,20)

In [326]:
pred_future = pd.DataFrame({'pre1' : pre1,
 'pre2':pre2,
 'pre3':pre3,
 'pre4':pre4,               
 'pre5':pre5,
 'pre6':pre6,
 'pre7':pre7,
 'pre8':pre8,
 'pre9':pre9,
 'pre10':pre10,
 'pre11':pre11,
 'pre12':pre12,
 'pre13':pre13,
 'pre14':pre14,
 'pre15':pre15,
 'pre16':pre16,
 'pre17':pre17,
 'pre18':pre18,
 'pre19':pre19,
 'pre20':pre20})
pred_future = pred_future.ix[:,[0,11,13,14,15,16,17,18,19,1,2,3,4,5,6,7,8,9,10,12]]
pred_future.columns=['zone1','zone2','zone3','zone4','zone5','zone6','zone7','zone8','zone9','zone10','zone11','zone12','zone13','zone14','zone15','zone16','zone17','zone18','zone19','zone20']

In [324]:
b = data_test

In [332]:
from sklearn.metrics import mean_squared_error
temp = np.zeros((len(pred_future),1))
temp1 = np.zeros((len(pred_future),1))
for i in range(len(pred_future)):
    temp[i][0] = np.sum(pred_future.ix[i,:])
    temp1[i][0] = np.sum(b.ix[i,:])
pred_future = np.concatenate((pred_future,temp),axis=1)
b = np.concatenate((b,temp1),axis=1)
rsme = np.sqrt(mean_squared_error(b, pred_future))
print('rsme:',rsme)

rsme: 50456.8166504
